<a href="https://colab.research.google.com/github/DIPANJAN001/Dipanjanpatra/blob/master/Soumya.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing training data 


In [ ]:
import pandas as pd
weather_data=pd.read_excel("/content/pv_02.xlsx") 

## Data format

In [ ]:
weather_data.head()

In [ ]:
weather_data.info()

## Weather input

In [ ]:
weather_input1=weather_data.drop('power_normed',axis=1)
weather_input=weather_input1.drop('time_idx',axis=1)
weather_input.head()

In [ ]:
solpow=weather_data['power_normed']
solpow

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(weather_input,solpow, test_size=0.1, random_state=42,shuffle=False)

In [ ]:
X_train.shape

In [ ]:
from tensorflow import keras 

In [ ]:
!pip install --upgrade tensorflow
!pip install --upgrade tensorflow-gpu

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
MLP_model = Sequential()
MLP_model.add(Dense(8, kernel_initializer='normal',input_dim =X_train.shape[1], activation='relu'))


MLP_model.add(Dense(32, kernel_initializer='normal',activation='relu'))
MLP_model.add(Dense(64, kernel_initializer='normal',activation='relu'))
MLP_model.add(Dense(16, kernel_initializer='normal',activation='relu'))

MLP_model.add(Dense(1, kernel_initializer='normal',activation='linear'))


MLP_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
MLP_model.summary()

In [ ]:
history=MLP_model.fit(X_train,y_train, epochs=80, batch_size=64, validation_split = 0.2)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
#plt.plot(history.history['mean_absolute_error'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['mean_absolute_error'])

## Importing test data

In [ ]:
import pandas as pd
weather_data_test=pd.read_excel("/content/pv_21.xlsx")

In [ ]:
weather_inputt=weather_data_test.drop('power_normed',axis=1)
weather_input_ts=weather_inputt.drop('time_idx',axis=1)
solpow=weather_data_test['power_normed']

In [ ]:
Ypred=MLP_model.predict(weather_input_ts)

In [ ]:
weather_data_test['Predicted_power_normed']=Ypred

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
fig = make_subplots(rows=1, cols=1)
fig.add_trace(
    go.Scatter(x=weather_data_test.reset_index()['time_idx'], y=weather_data_test.reset_index()['power_normed'],name='Actual generated solar power'),row=1,col=1
)
fig.add_trace(
    go.Scatter(x=weather_data_test.reset_index()['time_idx'], y=weather_data_test.reset_index()['Predicted_power_normed'],name='Predicted generated solar power by MLP with RMSE'),row=1,col=1
)


### Processing our Training data for feeding in LSTM network

In [ ]:
def lstm_data_transform(x_data, y_data, num_steps=5):
    """ Changes data to the format for LSTM training 
for sliding window approach """
    # Prepare the list for the transformed data
    X, y = list(), list()
    # Loop of the entire data set
    for i in range(x_data.shape[0]):
        # compute a new (sliding window) index
        end_ix = i + num_steps
        # if index is larger than the size of the dataset, we stop
        if end_ix >= x_data.shape[0]:
            break
        # Get a sequence of data for x
        seq_X = x_data[i:end_ix]
        # Get only the last element of the sequency for y
        seq_y = y_data[end_ix]
        # Append the list with sequencies
        X.append(seq_X)
        y.append(seq_y)
    # Make final arrays
    x_array = np.array(X)
    y_array = np.array(y)
    return x_array, y_array

### LSTM

In [ ]:
import numpy as np

In [ ]:
num_steps = 3
# training set
(x_train_transformed,
 y_train_transformed) = lstm_data_transform(X_train, y_train, num_steps=num_steps)
assert x_train_transformed.shape[0] == y_train_transformed.shape[0]
# test set
(x_test_transformed,
 y_test_transformed) = lstm_data_transform(weather_input_ts, solpow, num_steps=num_steps)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,SimpleRNN
from keras.layers import LSTM
from keras.layers import Dropout


### Building first LSTM Model for forecasting

In [ ]:
regressor = Sequential()
regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (3,49)))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.25))

regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.25))

regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.25))

regressor.add(Dense(units = 1))

regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

history=regressor.fit(x_train_transformed, y_train_transformed, epochs = 100, batch_size = 32)

In [ ]:
regressor.summary()

In [ ]:
plt.plot(history.history['loss'])
#plt.plot(history.history['val_loss'])
#plt.plot(history.history['mean_absolute_error'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
yp=regressor.predict(x_test_transformed)

In [ ]:
weather_data_test_lstm=weather_data_test.tail(4947)

In [ ]:
weather_data_test_lstm['Predicted_lstm']=yp

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
fig = make_subplots(rows=1, cols=1)
fig.add_trace(
    go.Scatter(x=weather_data_test_lstm.reset_index()['time_idx'], y=weather_data_test_lstm.reset_index()['power_normed'],name='Actual generated solar power'),row=1,col=1
)
fig.add_trace(
    go.Scatter(x=weather_data_test_lstm.reset_index()['time_idx'], y=weather_data_test_lstm.reset_index()['Predicted_lstm'],name='Predicted generated solar power LSTM '),row=1,col=1
)
fig.update_xaxes(title_text="Time")
fig.update_yaxes(title_text="SolarPower")

fig.show()

In [ ]:
rmse_LSTM= sqrt(mean_squared_error(weather_data_test_lstm['power_normed'],weather_data_test_lstm['Predicted_lstm']))